In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.io as scio
import linearDecode
import SoftmaxRe
from sklearn.preprocessing import OneHotEncoder, scale
from scipy import signal

In [2]:
path = 'F:\\UFLDL\\stlSubset'
data_test = scio.loadmat(path + '\\stlTestSubset.mat')
data_train = scio.loadmat(path + '\\stlTrainSubset.mat')

x_train, x_test, y_train, y_test = data_train['trainImages'], data_test['testImages'], \
                                   data_train['trainLabels'], data_test['testLabels']

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def ZCAwhiten(x):
    a, _, c, m = x.shape
    x = x.reshape(-1, m)
    x_mean = np.mean(x, 1).reshape(-1, 1)
    x -= x_mean
    # inn = x.T @ x
    # w, v = np.linalg.eig(inn)
    u, w, vh = np.linalg.svd(x)
    
    epsilon = np.ones((u.shape[0])) * 1e-6
    epsilon[:w.shape[0]] = w
    x_whiten = u / epsilon @ u.T @ x 
    return x_whiten.reshape(a, a, c, m)

In [6]:
x_train, x_test = ZCAwhiten(x_train), ZCAwhiten(x_test)
# x_train = scale(x_train.reshape(-1, x_train.shape[-1]).T).reshape(64, 64, 3, -1)
# x_test = scale(x_test.reshape(-1, x_test.shape[-1]).T).reshape(64, 64, 3, -1)

In [7]:
w_ld = np.load('w_linearDecode.npy')
b_ld, w_ld = w_ld[0], w_ld[1:]
kernelSize2, featureNum = w_ld.shape
kernelSize = np.sqrt(kernelSize2/3)
w_ld = w_ld.T.reshape(25, 3, 8, 8, order='C')
channelNum = 3
img_width, _, _, train_num = x_train.shape

In [8]:
h = np.zeros((img_width, img_width, featureNum, train_num))
for t in range(train_num):
    if t % 100 == 0:
        print(t)
    img = x_train[:, :, :, t]
    for f in range(featureNum):
        w = np.rot90(w_ld[f], 2)
        res = cv2.filter2D(img, ddepth=-1, kernel=w.T)
        res = np.sum(res, -1) + b_ld[f]
        h[:, :, f, t] = sigmoid(res)

0


100


200


300


400


...

1500


1600


1700


1800


1900


In [9]:
left, right = int(kernelSize // 2), int(img_width - kernelSize // 2)
print(kernelSize, left, right)
h = h[left-1:right, left-1:right, :, :]
print(h.shape)
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train).toarray()

8.0 4 60
(57, 57, 25, 2000)


In [10]:
def avePool(x, size):
    fil = np.ones((size, size)) / size**2
    img_width, _, channelNum, imgNum = x.shape
    y_width = img_width // size
    y = np.zeros((y_width, y_width, channelNum, imgNum))
    for i in range(y_width):
        for j in range(y_width):
            tmp = x[i*size : (i+1)*size, j*size : (j+1)*size, :, :]
            tmp = np.mean(tmp, 0)
            tmp = np.mean(tmp, 0)
            y[i, j] = tmp

    return y

In [11]:
h = avePool(h, size=19)

In [33]:
x_smx = h.reshape(-1, train_num).T
x_smx = scale(x_smx)
w_smx = SoftmaxRe.softmax_regression(x_smx, y_train, 
                                     alpha=3e-4, n_iters=4  00, lambd=1e-3)

0  :  2.85903876286
5  :  1.79237389774
10  :  1.68688229372
15  :  1.62531924966
20  :  1.5819596494
25  :  1.54880832517
30  :  1.52226617524
35  :  1.50030335249


40  :  1.4817376931
45  :  1.46573163245
50  :  1.45177364918
55  :  1.43941763952
60  :  1.42842360901
65  :  1.41850024121
70  :  1.40955246591


75  :  1.40134536466
80  :  1.39388742786
85  :  1.38694331713
90  :  1.3806221362
95  :  1.37464115741
100  :  1.36923357779


105  :  1.36401745475
110  :  1.35939972856
115  :  1.35483267761
120  :  1.35097447924
125  :  1.34702020221
130  :  1.3439717579
135  : 

 1.34063736829
140  :  1.33842150597
145  :  1.33559866265
150  :  1.3339796762
155  :  1.33130824133
160  :  1.32983198346
165  :  1.32698947427


...

820  :  1.22457300184
825  :  1.22413404699
830  :  1.22414868716
835  :  1.22371742955
840  :  1.22373488554
845  :  1.22331108465
850  :  1.22333127048
855  :  1.22291469476


860  :  1.22293752824
865  :  1.2225279547
870  :  1.22255335725
875  :  1.22215057113
880  :  1.22217846752
885  :  1.22178226187
890  :  1.22181258008
895  :  1.22142275547


900  :  1.22145542648
905  :  1.22107179065
910  :  1.2211067483
915  :  1.22072911585
920  :  1.22076629665
925  :  1.22039448879
930  :  1.22043383181
935  :  1.22006767602


940  :  1.22010912272
945  :  1.21974845256
950  :  1.2197919467
955  :  1.21943660148
960  :  1.21948208896
965  :  1.21913191359
970  :  1.21917934238
975  :  1.21883418705


980  :  1.21888350707
985  :  1.21854322712
990  :  1.21859439012
995  :  1.21825884579
ACC_train: 0.6165


In [26]:
h.shape


(3, 3, 25, 2000)